In [1]:
import sys
import pytest
import os
import adapt
from adapt.driver import *
from adapt.pyscf_backend import *
from adapt.of_translator import *

separation=1.8

"""Test ADAPT on H4."""
if os.path.exists('test') == False:
    os.makedirs('test')
# geom = 'H 0 0 0; H 0 0 1; H 0 0 2; H 0 0 3'

# geom=f'Be 0 0 0; H 0 0 -{separation}; H 0 0 {separation}'

geom=f'Li 0 0 0; H 0 0 {separation}'

basis = "sto-3g"
reference = "rhf"

#Compute molecular integrals, etc.
E_nuc, H_core, g, D, C, hf_energy = get_integrals(geom, basis, reference, chkfile = "scr.chk", read = False)

#Compute number of electrons
N_e = int(np.trace(D))

#Compute sparse matrix reps
H, ref, N_qubits, S2, Sz, Nop = of_from_arrays(E_nuc, H_core, g, N_e)





******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = atom
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = 1e-14
SCF max_cycles = 10000
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scr.chk
max_memory 8000 MB (current use 267 MB)
Initial guess from superposition of atomic densities.
Spherically averaged atomic HF for {'H', 'Li'}


******** <class 'pyscf.scf.atom_hf.AtomSphAverageRHF'> ********
method = AtomSphAverageRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpo2s2g4jh
max_memory 8000 MB (current use 269 MB)
atom = Li
Set gradient conv threshold to 3.16228e-05
Initial g

In [2]:
#Build system data and get pool from it
s = sm.system_data(H, ref, N_e, N_qubits)
pool, v_pool = s.uccsd_pool(approach = 'vanilla')

ansatz=list(range(len(pool)))
params=np.zeros(len(ansatz))

for i, label in enumerate(v_pool):
    print(i, label)

state=t_ucc_state(params, ansatz, pool, ref)
E_init=t_ucc_E(params, ansatz, H, pool, ref)
print("Initial t-UCC energy:", E_init)

Operators in pool:
92
0 0->4
1 0,1->4,5
2 0,1->4,7
3 0,1->4,9
4 0,1->4,11
5 0,2->4,6
6 0,2->4,8
7 0,2->4,10
8 0,3->4,5
9 0,3->4,7
10 0,3->4,9
11 0,3->4,11
12 0,1->5,6
13 0,1->5,8
14 0,1->5,10
15 0,3->5,6
16 0,3->5,8
17 0,3->5,10
18 0->6
19 0,1->6,7
20 0,1->6,9
21 0,1->6,11
22 0,2->6,8
23 0,2->6,10
24 0,3->6,7
25 0,3->6,9
26 0,3->6,11
27 0,1->7,8
28 0,1->7,10
29 0,3->7,8
30 0,3->7,10
31 0->8
32 0,1->8,9
33 0,1->8,11
34 0,2->8,10
35 0,3->8,9
36 0,3->8,11
37 0,1->9,10
38 0,3->9,10
39 0->10
40 0,1->10,11
41 0,3->10,11
42 1,2->4,5
43 1,2->4,7
44 1,2->4,9
45 1,2->4,11
46 1->5
47 1,2->5,6
48 1,2->5,8
49 1,2->5,10
50 1,3->5,7
51 1,3->5,9
52 1,3->5,11
53 1,2->6,7
54 1,2->6,9
55 1,2->6,11
56 1->7
57 1,2->7,8
58 1,2->7,10
59 1,3->7,9
60 1,3->7,11
61 1,2->8,9
62 1,2->8,11
63 1->9
64 1,2->9,10
65 1,3->9,11
66 1,2->10,11
67 1->11
68 2->4
69 2,3->4,5
70 2,3->4,7
71 2,3->4,9
72 2,3->4,11
73 2,3->5,6
74 2,3->5,8
75 2,3->5,10
76 2->6
77 2,3->6,7
78 2,3->6,9
79 2,3->6,11
80 2,3->7,8
81 2,3->7,10
82 2->8


In [3]:

#Build 'xiphos' object (essentially ADAPT class)
xiphos = Xiphos(H, ref, "test", pool, v_pool, sym_ops = {"H": H, "S_z": Sz, "S^2": S2, "N": Nop})
params = np.array([])

Restarting the calculation.

---------------------------

Reference information:
     H:  -7.8500186971668144
   S_z:   0.0000000000000000
   S^2:   0.0000000000000000
     N:   4.0000000000000000

ED information:
ED Solution 1:
     H:  -7.8745240249913220
   S_z:  -0.0000000000000000
   S^2:   0.0000000000000000
     N:   4.0000000000000027
ED Solution 2:
     H:  -7.8006677434425038
   S_z:  -0.4637859610507805
   S^2:   0.7500000000000013
     N:   5.0000000000000098
ED Solution 3:
     H:  -7.8006677434424763
   S_z:   0.4637859610507788
   S^2:   0.7499999999999988
     N:   4.9999999999999920
ED Solution 4:
     H:  -7.7734347398104200
   S_z:  -0.0001862622526618
   S^2:   2.0000000000000022
     N:   4.0000000000000044
ED Solution 5:
     H:  -7.7734347398104156
   S_z:  -0.9998137377473389
   S^2:   2.0000000000000013
     N:   4.0000000000000027
ED Solution 6:
     H:  -7.7734347398104156
   S_z:   1.0000000000000009
   S^2:   2.0000000000000018
     N:   4.0000000000000036


In [ ]:
error =xiphos.gd_adapt(params, ansatz, ref)
print("Final energy:", error)

Diagonalizing operator...
Operator diagonalized in 48.904502868652344 s
Diagonalizing operator...
Operator diagonalized in 36.3305983543396 s
Diagonalizing operator...
Operator diagonalized in 36.810763359069824 s
Diagonalizing operator...
Operator diagonalized in 37.18565058708191 s
Diagonalizing operator...
Operator diagonalized in 47.83239722251892 s
Diagonalizing operator...
Operator diagonalized in 42.140244483947754 s
Diagonalizing operator...


In [ ]:
import argparse
import numpy as np

# adapt package internals
from adapt.pyscf_backend import get_integrals
from adapt.of_translator import of_from_arrays
import adapt.system_methods as sm
from adapt.driver import t_ucc_E

def build_system(geom: str, basis: str = "sto-3g", reference: str = "rhf"):
    E_nuc, H_core, g, D, C, hf_energy = get_integrals(geom, basis, reference, chkfile="scr.chk", read=False)
    N_e = int(np.trace(D))
    H, ref, N_qubits, S2, Sz, Nop = of_from_arrays(E_nuc, H_core, g, N_e)
    return H, ref, N_qubits, N_e

def main():
    p = argparse.ArgumentParser(description="Simple full-UCCSD VQE using adapt internals (default: BeH / sto-3g, optimize with L-BFGS-B)")
    p.add_argument("--geom", type=str, default="Be 0 0 0; H 0 0 0.74; H 0 0 -0.74", help="Molecule geometry string (PSI/PySCF style)")
    p.add_argument("--basis", type=str, default="sto-3g")
    p.add_argument("--ref", type=str, default="rhf")
    p.add_argument("--no-optimize", action="store_true", help="Do not run the classical optimizer (run energy eval only)")
    p.add_argument("--seed", type=int, default=42)
    # ignore extra Jupyter args when running inside notebooks
    args, _ = p.parse_known_args()

    H, ref, N_qubits, N_e = build_system(args.geom, basis=args.basis, reference=args.ref)

    s = sm.system_data(H, ref, N_e, N_qubits)
    jw_pool, v_pool = s.uccsd_pool(approach="vanilla")

    # full-UCCSD ansatz: use every operator in the pool in sequence
    ansatz = list(range(len(jw_pool)))
    params = np.zeros(len(ansatz))

    # evaluate initial energy
    E0 = t_ucc_E(params, ansatz, H, jw_pool, ref)
    print(f"Initial (params=0) energy: {float(E0):.12f} Ha  |  #operators = {len(ansatz)}")

    if args.no_optimize:
        return

    try:
        from scipy.optimize import minimize
    except Exception:
        print("scipy not available: cannot optimize. Install scipy and retry.")
        return

    rng = np.random.default_rng(args.seed)
    # small random start (better than all-zeros to break possible stationary points)
    x0 = rng.normal(scale=0.05, size=len(ansatz))

    def obj(x):
        return float(t_ucc_E(x, ansatz, H, jw_pool, ref))

    def mycallback(xk):
        e = obj(xk)
        print(f"Current energy: {e:.12f} Ha")
    res = minimize(obj, x0, method="L-BFGS-B", options={"maxiter": 200},callback=mycallback)
    print("Optimizer success:", bool(res.success), res.message)
    print(f"Optimized energy: {res.fun:.12f} Ha")
    if len(res.x) > 0:
        print("First 8 parameters:", np.array2string(res.x[:8], precision=6, separator=", "))

if __name__ == "__main__":
    main()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = atom
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = 1e-14
SCF max_cycles = 10000
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scr.chk
max_memory 8000 MB (current use 338 MB)


Initial guess from superposition of atomic densities.
Spherically averaged atomic HF for {'Be', 'H'}


******** <class 'pyscf.scf.atom_hf.AtomSphAverageRHF'> ********
method = AtomSphAverageRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpjeipu8wr
max_memory 8000 MB (current use 338 MB)
atom = Be
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -14.3355292058976
cycle= 1 E= -14.351880476202  delta_E= -0.0164  |g|=    0  |ddm|= 0.0518
cycle= 2 E= -14.351880476202  delta_E= -1.78e-15  |g|=    0  |ddm|= 1.1e-15
Extra cycle  E= -14.351880476202  delta_E=    0  |g|=    0  |ddm|=    0
Atomic HF for atom  Be  converged. SCF energy = -14.351880476202



******** 1 electron system ********
Atomic HF for atom  H  converge